In [1]:

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
#from ProcessTrueStateActionData import read_df_in_chunks

2023-03-30 09:38:03.792371: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 09:38:04.010853: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-30 09:38:04.859930: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/adamprice/miniconda3/envs/u75a/lib/
2023-03-30 09:38:04.860035: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_

### Load and preprocess the data 
(produce tf train+test datasets)

In [3]:
train_test_split = 0.95

data_path = 'logs/PPO/no_decoy_200000/data'
states = np.load(data_path + '/states.npy')
states_t = np.load(data_path + '/states_t.npy')
afterstates = np.load(data_path + '/afterstates.npy')
next_states = np.load(data_path + '/next_states.npy')
next_states_t = np.load(data_path + '/next_states_t.npy')
actions = np.load(data_path + '/actions_onehot.npy')
rewards = np.load(data_path + '/rewards.npy')

states_actions = np.concatenate([states, actions], axis=1)
states_actions_t = np.concatenate([states_t, actions], axis=1)
next_states_delta = next_states - states

afterstates = np.array(afterstates, dtype=np.float32)
next_states = np.array(next_states, dtype=np.float32)
states_actions = np.array(states_actions, dtype=np.float32)

divide = int(train_test_split * afterstates.shape[0])
afterstates_train = afterstates[:divide,:]
next_states_train = next_states[:divide,:]
afterstates_test = afterstates[divide:,:]
next_states_test = next_states[divide:,:]
print(afterstates.shape[0] - divide)

FileNotFoundError: [Errno 2] No such file or directory: 'logs/APPO/TrueStates_200_4000_B_Line_nodecoys/data/states.npy'

In [4]:
ACTION_LEN = 41
STATE_LEN = 91

In [73]:
afterstates.shape[0]

198000

In [3]:
states_seq = np.load(data_path + '/states_seq.npy')
actions_seq = np.load(data_path + '/actions_onehot_seq.npy')

states_actions_seq = np.concatenate([states_seq, actions_seq], axis=2)

FileNotFoundError: [Errno 2] No such file or directory: 'logs/APPO/TrueStates_200_4000_B_Line_nodecoys/data/states_seq.npy'

In [5]:
states_seq_10 = np.load(data_path + '/states_seq_10.npy')
actions_seq_10 = np.load(data_path + '/actions_onehot_seq_10.npy')

states_actions_seq_10 = np.concatenate([states_seq_10, actions_seq_10], axis=2)

In [ ]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input
from keras.layers import Bidirectional
from keras.models import Model

NUM_NODES = 6
NODE_CLASSES = [3, 4]

data_map = {}
losses = []
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        data_map[str(i)+str(n)] = next_states[:,index:index+n]
        losses.append(tf.keras.losses.CategoricalCrossentropy())
        index += n
        
input_ = Input(shape=(42+20,))
x = Dense(512, activation='relu')(input_)
x = Dense(512, activation='relu')(x)

outs = []
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        x_ = Dense(128, activation='relu')(x)
        outs.append(Dense(n, activation='softmax', name=str(i)+str(n))(x_))

model = Model(input_, outs)
model.compile(optimizer='adam', loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])
callback = tf.keras.callbacks.EarlyStopping(monitor='54_categorical_accuracy', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions, data_map, epochs=50, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=128)

In [7]:
## from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input
from keras.layers import Bidirectional
from keras.models import Model

NUM_NODES = 13
NODE_CLASSES = [3, 4]

data_map = {}
losses = []
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        #data_map[str(i)+str(n)] = next_states[:,index:index+n]
        losses.append(tf.keras.losses.CategoricalCrossentropy())
        index += n
        
input_ = Input(shape=(91+41+1,))
#x = Dense(512, activation='relu')(input_)
#x = Dense(512, activation='relu')(x)
outs = []
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        x_ = Dense(128, activation='relu')(input_)
        outs.append(Dense(n, activation='softmax', name=str(i)+str(n))(x_))

model = Model(input_, outs)
model.compile(optimizer='adam', loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])

model.summary()
#callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

#from keras import backend as K
#K.set_value(model.optimizer.learning_rate, 0.0005)

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

#model.fit(states_actions_t, data_map, epochs=200, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=128)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 133)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          17152       ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 128)          17152       ['input_1[0][0]']                
                                                                                                  
 dense_2 (Dense)                (None, 128)          17152       ['input_1[0][0]']                
                                                                                              

 64 (Dense)                     (None, 4)            516         ['dense_13[0][0]']               
                                                                                                  
 73 (Dense)                     (None, 3)            387         ['dense_14[0][0]']               
                                                                                                  
 74 (Dense)                     (None, 4)            516         ['dense_15[0][0]']               
                                                                                                  
 83 (Dense)                     (None, 3)            387         ['dense_16[0][0]']               
                                                                                                  
 84 (Dense)                     (None, 4)            516         ['dense_17[0][0]']               
                                                                                                  
 93 (Dense

In [7]:
#1.56
model.save_weights('AfterStateModel_MulitLabel')

In [9]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input, GRU
from keras.layers import Bidirectional
from keras.models import Model

NUM_NODES = 13
NODE_CLASSES = [3, 4]

data_map = {}
losses = []
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        data_map[str(i)+str(n)] = next_states[:,index:index+n]
        losses.append(tf.keras.losses.CategoricalCrossentropy())
        index += n
        
input_ = Input(shape=(10,STATE_LEN+ACTION_LEN,))
#x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(input_)
#x = Bidirectional(GRU(64))(input_)
x = Bidirectional(LSTM(256))(input_)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)

outs = []
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        x_ = Dense(256, activation='relu')(x)
        outs.append(Dense(n, activation='softmax', name=str(i)+str(n))(x_))

model = Model(input_, outs)
model.compile(optimizer='adam', loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.005)

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions_seq_10, data_map, epochs=1000, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=256)

Epoch 1/1000
697/697 [==============================] - 58s 47ms/step - loss: 8.1866 - 03_loss: 0.0029 - 04_loss: 0.0035 - 13_loss: 0.3314 - 14_loss: 0.1877 - 23_loss: 0.4319 - 24_loss: 0.5159 - 33_loss: 0.8464 - 34_loss: 1.0939 - 43_loss: 0.0022 - 44_loss: 0.0033 - 53_loss: 0.0023 - 54_loss: 0.0034 - 63_loss: 0.0026 - 64_loss: 0.0031 - 73_loss: 0.2563 - 74_loss: 0.2695 - 83_loss: 0.2401 - 84_loss: 0.0032 - 93_loss: 0.2629 - 94_loss: 0.7452 - 103_loss: 0.2714 - 104_loss: 0.7581 - 113_loss: 0.2622 - 114_loss: 0.7306 - 123_loss: 0.2656 - 124_loss: 0.6913 - 03_categorical_accuracy: 0.9986 - 04_categorical_accuracy: 0.9986 - 13_categorical_accuracy: 0.9149 - 14_categorical_accuracy: 0.9567 - 23_categorical_accuracy: 0.8844 - 24_categorical_accuracy: 0.8692 - 33_categorical_accuracy: 0.6544 - 34_categorical_accuracy: 0.4618 - 43_categorical_accuracy: 0.9998 - 44_categorical_accuracy: 0.9987 - 53_categorical_accuracy: 0.9994 - 54_categorical_accuracy: 0.9986 - 63_categorical_accuracy: 0.9986

Epoch 4/1000
697/697 [==============================] - 29s 42ms/step - loss: 7.7047 - 03_loss: 4.2733e-09 - 04_loss: 4.2800e-09 - 13_loss: 0.3133 - 14_loss: 0.1654 - 23_loss: 0.4084 - 24_loss: 0.4827 - 33_loss: 0.8128 - 34_loss: 1.0347 - 43_loss: 7.4587e-07 - 44_loss: 3.2177e-09 - 53_loss: 6.7539e-09 - 54_loss: 9.7548e-09 - 63_loss: 3.4539e-09 - 64_loss: 5.3002e-09 - 73_loss: 0.2469 - 74_loss: 0.2467 - 83_loss: 0.2344 - 84_loss: 2.9535e-09 - 93_loss: 0.2534 - 94_loss: 0.7007 - 103_loss: 0.2613 - 104_loss: 0.7081 - 113_loss: 0.2527 - 114_loss: 0.6796 - 123_loss: 0.2552 - 124_loss: 0.6483 - 03_categorical_accuracy: 1.0000 - 04_categorical_accuracy: 1.0000 - 13_categorical_accuracy: 0.9162 - 14_categorical_accuracy: 0.9581 - 23_categorical_accuracy: 0.8858 - 24_categorical_accuracy: 0.8704 - 33_categorical_accuracy: 0.6579 - 34_categorical_accuracy: 0.4888 - 43_categorical_accuracy: 1.0000 - 44_categorical_accuracy: 1.0000 - 53_categorical_accuracy: 1.0000 - 54_categorical_accuracy: 1.00

Epoch 7/1000
697/697 [==============================] - 29s 41ms/step - loss: 7.5022 - 03_loss: 4.8232e-10 - 04_loss: 5.2045e-10 - 13_loss: 0.3109 - 14_loss: 0.1587 - 23_loss: 0.4020 - 24_loss: 0.4691 - 33_loss: 0.8018 - 34_loss: 1.0145 - 43_loss: 1.6056e-07 - 44_loss: 4.0740e-10 - 53_loss: 5.4186e-10 - 54_loss: 8.0878e-10 - 63_loss: 3.6927e-10 - 64_loss: 5.5591e-10 - 73_loss: 0.2435 - 74_loss: 0.2329 - 83_loss: 0.2336 - 84_loss: 4.0740e-10 - 93_loss: 0.2525 - 94_loss: 0.6749 - 103_loss: 0.2603 - 104_loss: 0.6739 - 113_loss: 0.2514 - 114_loss: 0.6512 - 123_loss: 0.2545 - 124_loss: 0.6165 - 03_categorical_accuracy: 1.0000 - 04_categorical_accuracy: 1.0000 - 13_categorical_accuracy: 0.9168 - 14_categorical_accuracy: 0.9586 - 23_categorical_accuracy: 0.8866 - 24_categorical_accuracy: 0.8702 - 33_categorical_accuracy: 0.6597 - 34_categorical_accuracy: 0.4969 - 43_categorical_accuracy: 1.0000 - 44_categorical_accuracy: 1.0000 - 53_categorical_accuracy: 1.0000 - 54_categorical_accuracy: 1.00

Epoch 10/1000
697/697 [==============================] - 29s 42ms/step - loss: 7.3483 - 03_loss: 0.0000e+00 - 04_loss: 0.0000e+00 - 13_loss: 0.3086 - 14_loss: 0.1532 - 23_loss: 0.3968 - 24_loss: 0.4575 - 33_loss: 0.7942 - 34_loss: 1.0002 - 43_loss: 2.6009e-09 - 44_loss: 0.0000e+00 - 53_loss: 0.0000e+00 - 54_loss: 0.0000e+00 - 63_loss: 0.0000e+00 - 64_loss: 0.0000e+00 - 73_loss: 0.2404 - 74_loss: 0.2219 - 83_loss: 0.2330 - 84_loss: 0.0000e+00 - 93_loss: 0.2514 - 94_loss: 0.6544 - 103_loss: 0.2590 - 104_loss: 0.6496 - 113_loss: 0.2504 - 114_loss: 0.6298 - 123_loss: 0.2534 - 124_loss: 0.5946 - 03_categorical_accuracy: 1.0000 - 04_categorical_accuracy: 1.0000 - 13_categorical_accuracy: 0.9169 - 14_categorical_accuracy: 0.9591 - 23_categorical_accuracy: 0.8871 - 24_categorical_accuracy: 0.8708 - 33_categorical_accuracy: 0.6623 - 34_categorical_accuracy: 0.5053 - 43_categorical_accuracy: 1.0000 - 44_categorical_accuracy: 1.0000 - 53_categorical_accuracy: 1.0000 - 54_categorical_accuracy: 1.0

Epoch 13/1000
697/697 [==============================] - 28s 41ms/step - loss: 7.2352 - 03_loss: 0.0000e+00 - 04_loss: 0.0000e+00 - 13_loss: 0.3055 - 14_loss: 0.1489 - 23_loss: 0.3924 - 24_loss: 0.4481 - 33_loss: 0.7879 - 34_loss: 0.9893 - 43_loss: 0.0000e+00 - 44_loss: 0.0000e+00 - 53_loss: 0.0000e+00 - 54_loss: 0.0000e+00 - 63_loss: 0.0000e+00 - 64_loss: 0.0000e+00 - 73_loss: 0.2373 - 74_loss: 0.2147 - 83_loss: 0.2320 - 84_loss: 0.0000e+00 - 93_loss: 0.2496 - 94_loss: 0.6412 - 103_loss: 0.2576 - 104_loss: 0.6343 - 113_loss: 0.2490 - 114_loss: 0.6154 - 123_loss: 0.2516 - 124_loss: 0.5805 - 03_categorical_accuracy: 1.0000 - 04_categorical_accuracy: 1.0000 - 13_categorical_accuracy: 0.9174 - 14_categorical_accuracy: 0.9590 - 23_categorical_accuracy: 0.8873 - 24_categorical_accuracy: 0.8710 - 33_categorical_accuracy: 0.6634 - 34_categorical_accuracy: 0.5099 - 43_categorical_accuracy: 1.0000 - 44_categorical_accuracy: 1.0000 - 53_categorical_accuracy: 1.0000 - 54_categorical_accuracy: 1.0

...

Epoch 121/1000
697/697 [==============================] - 29s 41ms/step - loss: 6.6712 - 03_loss: 0.0000e+00 - 04_loss: 0.0000e+00 - 13_loss: 0.2664 - 14_loss: 0.1272 - 23_loss: 0.3484 - 24_loss: 0.3982 - 33_loss: 0.7375 - 34_loss: 0.9373 - 43_loss: 0.0000e+00 - 44_loss: 0.0000e+00 - 53_loss: 0.0000e+00 - 54_loss: 0.0000e+00 - 63_loss: 0.0000e+00 - 64_loss: 0.0000e+00 - 73_loss: 0.2051 - 74_loss: 0.1940 - 83_loss: 0.2065 - 84_loss: 0.0000e+00 - 93_loss: 0.2223 - 94_loss: 0.6133 - 103_loss: 0.2291 - 104_loss: 0.6005 - 113_loss: 0.2224 - 114_loss: 0.5864 - 123_loss: 0.2244 - 124_loss: 0.5522 - 03_categorical_accuracy: 1.0000 - 04_categorical_accuracy: 1.0000 - 13_categorical_accuracy: 0.9173 - 14_categorical_accuracy: 0.9594 - 23_categorical_accuracy: 0.8869 - 24_categorical_accuracy: 0.8709 - 33_categorical_accuracy: 0.6720 - 34_categorical_accuracy: 0.5357 - 43_categorical_accuracy: 1.0000 - 44_categorical_accuracy: 1.0000 - 53_categorical_accuracy: 1.0000 - 54_categorical_accuracy: 1.

Epoch 124/1000
697/697 [==============================] - 28s 40ms/step - loss: 6.7021 - 03_loss: 0.0000e+00 - 04_loss: 0.0000e+00 - 13_loss: 0.2686 - 14_loss: 0.1290 - 23_loss: 0.3502 - 24_loss: 0.4011 - 33_loss: 0.7415 - 34_loss: 0.9407 - 43_loss: 0.0000e+00 - 44_loss: 0.0000e+00 - 53_loss: 0.0000e+00 - 54_loss: 0.0000e+00 - 63_loss: 0.0000e+00 - 64_loss: 0.0000e+00 - 73_loss: 0.2074 - 74_loss: 0.1946 - 83_loss: 0.2074 - 84_loss: 0.0000e+00 - 93_loss: 0.2228 - 94_loss: 0.6146 - 103_loss: 0.2307 - 104_loss: 0.6025 - 113_loss: 0.2234 - 114_loss: 0.5878 - 123_loss: 0.2266 - 124_loss: 0.5533 - 03_categorical_accuracy: 1.0000 - 04_categorical_accuracy: 1.0000 - 13_categorical_accuracy: 0.9171 - 14_categorical_accuracy: 0.9592 - 23_categorical_accuracy: 0.8870 - 24_categorical_accuracy: 0.8707 - 33_categorical_accuracy: 0.6719 - 34_categorical_accuracy: 0.5350 - 43_categorical_accuracy: 1.0000 - 44_categorical_accuracy: 1.0000 - 53_categorical_accuracy: 1.0000 - 54_categorical_accuracy: 1.

Epoch 127/1000
697/697 [==============================] - 28s 40ms/step - loss: 6.6688 - 03_loss: 0.0000e+00 - 04_loss: 0.0000e+00 - 13_loss: 0.2656 - 14_loss: 0.1268 - 23_loss: 0.3475 - 24_loss: 0.3980 - 33_loss: 0.7381 - 34_loss: 0.9374 - 43_loss: 0.0000e+00 - 44_loss: 0.0000e+00 - 53_loss: 0.0000e+00 - 54_loss: 0.0000e+00 - 63_loss: 0.0000e+00 - 64_loss: 0.0000e+00 - 73_loss: 0.2058 - 74_loss: 0.1939 - 83_loss: 0.2065 - 84_loss: 0.0000e+00 - 93_loss: 0.2222 - 94_loss: 0.6126 - 103_loss: 0.2292 - 104_loss: 0.6004 - 113_loss: 0.2219 - 114_loss: 0.5863 - 123_loss: 0.2249 - 124_loss: 0.5517 - 03_categorical_accuracy: 1.0000 - 04_categorical_accuracy: 1.0000 - 13_categorical_accuracy: 0.9168 - 14_categorical_accuracy: 0.9594 - 23_categorical_accuracy: 0.8872 - 24_categorical_accuracy: 0.8703 - 33_categorical_accuracy: 0.6723 - 34_categorical_accuracy: 0.5350 - 43_categorical_accuracy: 1.0000 - 44_categorical_accuracy: 1.0000 - 53_categorical_accuracy: 1.0000 - 54_categorical_accuracy: 1.

Epoch 130/1000
697/697 [==============================] - 28s 40ms/step - loss: 6.6646 - 03_loss: 0.0000e+00 - 04_loss: 0.0000e+00 - 13_loss: 0.2652 - 14_loss: 0.1270 - 23_loss: 0.3465 - 24_loss: 0.3972 - 33_loss: 0.7376 - 34_loss: 0.9378 - 43_loss: 0.0000e+00 - 44_loss: 0.0000e+00 - 53_loss: 0.0000e+00 - 54_loss: 0.0000e+00 - 63_loss: 0.0000e+00 - 64_loss: 0.0000e+00 - 73_loss: 0.2045 - 74_loss: 0.1938 - 83_loss: 0.2067 - 84_loss: 0.0000e+00 - 93_loss: 0.2212 - 94_loss: 0.6130 - 103_loss: 0.2287 - 104_loss: 0.6006 - 113_loss: 0.2215 - 114_loss: 0.5867 - 123_loss: 0.2247 - 124_loss: 0.5519 - 03_categorical_accuracy: 1.0000 - 04_categorical_accuracy: 1.0000 - 13_categorical_accuracy: 0.9170 - 14_categorical_accuracy: 0.9593 - 23_categorical_accuracy: 0.8868 - 24_categorical_accuracy: 0.8709 - 33_categorical_accuracy: 0.6723 - 34_categorical_accuracy: 0.5365 - 43_categorical_accuracy: 1.0000 - 44_categorical_accuracy: 1.0000 - 53_categorical_accuracy: 1.0000 - 54_categorical_accuracy: 1.

Epoch 133/1000
697/697 [==============================] - 28s 40ms/step - loss: 6.6669 - 03_loss: 0.0000e+00 - 04_loss: 0.0000e+00 - 13_loss: 0.2651 - 14_loss: 0.1270 - 23_loss: 0.3474 - 24_loss: 0.3980 - 33_loss: 0.7371 - 34_loss: 0.9376 - 43_loss: 0.0000e+00 - 44_loss: 0.0000e+00 - 53_loss: 0.0000e+00 - 54_loss: 0.0000e+00 - 63_loss: 0.0000e+00 - 64_loss: 0.0000e+00 - 73_loss: 0.2048 - 74_loss: 0.1938 - 83_loss: 0.2058 - 84_loss: 0.0000e+00 - 93_loss: 0.2216 - 94_loss: 0.6136 - 103_loss: 0.2280 - 104_loss: 0.6010 - 113_loss: 0.2214 - 114_loss: 0.5867 - 123_loss: 0.2253 - 124_loss: 0.5527 - 03_categorical_accuracy: 1.0000 - 04_categorical_accuracy: 1.0000 - 13_categorical_accuracy: 0.9171 - 14_categorical_accuracy: 0.9596 - 23_categorical_accuracy: 0.8868 - 24_categorical_accuracy: 0.8705 - 33_categorical_accuracy: 0.6720 - 34_categorical_accuracy: 0.5366 - 43_categorical_accuracy: 1.0000 - 44_categorical_accuracy: 1.0000 - 53_categorical_accuracy: 1.0000 - 54_categorical_accuracy: 1.

In [10]:
model.save_weights('NextStateModel_MulitLabel_rnn')

In [1]:
model.fit(states_actions_seq_10, data_map, epochs=100, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=256)

NameError: name 'model' is not defined

In [30]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input, GRU
from keras.layers import Bidirectional
from keras.models import Model

NUM_NODES = 6
NODE_CLASSES = [3, 4]

data_map = {}
losses = []
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        data_map[str(i)+str(n)] = next_states[:,index:index+n]
        losses.append(tf.keras.losses.CategoricalCrossentropy())
        index += n
        
input_ = Input(shape=(10,42+20,))
#x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(input_)
#x = Bidirectional(GRU(64))(input_)
x = Bidirectional(LSTM(256))(input_)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)

outs = []
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        x_ = Dense(256, activation='relu')(x)
        outs.append(Dense(n, activation='softmax', name=str(i)+str(n))(x_))

model = Model(input_, outs)
model.compile(optimizer='adam', loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions_seq_10, data_map, epochs=1000, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=256)

Epoch 1/1000
697/697 [==============================] - 32s 28ms/step - loss: 4.3925 - 03_loss: 0.6321 - 04_loss: 0.1746 - 13_loss: 0.7745 - 14_loss: 0.6850 - 23_loss: 0.0075 - 24_loss: 0.0088 - 33_loss: 0.0069 - 34_loss: 0.0112 - 43_loss: 0.2902 - 44_loss: 1.1840 - 53_loss: 0.2869 - 54_loss: 0.3309 - 03_categorical_accuracy: 0.6734 - 04_categorical_accuracy: 0.9531 - 13_categorical_accuracy: 0.6536 - 14_categorical_accuracy: 0.6531 - 23_categorical_accuracy: 0.9997 - 24_categorical_accuracy: 0.9998 - 33_categorical_accuracy: 0.9996 - 34_categorical_accuracy: 0.9986 - 43_categorical_accuracy: 0.9182 - 44_categorical_accuracy: 0.4848 - 53_categorical_accuracy: 0.9147 - 54_categorical_accuracy: 0.9054 - val_loss: 4.2576 - val_03_loss: 0.6201 - val_04_loss: 0.1571 - val_13_loss: 0.7662 - val_14_loss: 0.6684 - val_23_loss: 1.2516e-07 - val_24_loss: 3.7638e-07 - val_33_loss: 1.2050e-07 - val_34_loss: 1.7278e-07 - val_43_loss: 0.2729 - val_44_loss: 1.1825 - val_53_loss: 0.2771 - val_54_loss:

Epoch 7/1000
697/697 [==============================] - 16s 23ms/step - loss: 3.9325 - 03_loss: 0.6065 - 04_loss: 0.1270 - 13_loss: 0.7547 - 14_loss: 0.6422 - 23_loss: 5.6581e-09 - 24_loss: 7.4636e-09 - 33_loss: 5.7243e-09 - 34_loss: 6.0782e-09 - 43_loss: 0.2671 - 44_loss: 1.0405 - 53_loss: 0.2657 - 54_loss: 0.2289 - 03_categorical_accuracy: 0.6796 - 04_categorical_accuracy: 0.9648 - 13_categorical_accuracy: 0.6575 - 14_categorical_accuracy: 0.6712 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9209 - 44_categorical_accuracy: 0.5459 - 53_categorical_accuracy: 0.9180 - 54_categorical_accuracy: 0.9264 - val_loss: 4.3855 - val_03_loss: 0.6122 - val_04_loss: 0.1547 - val_13_loss: 0.7617 - val_14_loss: 0.6675 - val_23_loss: 1.3264e-08 - val_24_loss: 1.3516e-08 - val_33_loss: 1.3270e-08 - val_34_loss: 1.3288e-08 - val_43_loss: 0.2665 - val_44_loss: 1.2742 - val_53_loss: 0.273

Epoch 13/1000
697/697 [==============================] - 17s 24ms/step - loss: 3.5486 - 03_loss: 0.5763 - 04_loss: 0.0987 - 13_loss: 0.7320 - 14_loss: 0.6009 - 23_loss: 6.1411e-10 - 24_loss: 8.2751e-10 - 33_loss: 7.1847e-10 - 34_loss: 7.5860e-10 - 43_loss: 0.2463 - 44_loss: 0.8873 - 53_loss: 0.2474 - 54_loss: 0.1598 - 03_categorical_accuracy: 0.6986 - 04_categorical_accuracy: 0.9699 - 13_categorical_accuracy: 0.6626 - 14_categorical_accuracy: 0.6879 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9243 - 44_categorical_accuracy: 0.6109 - 53_categorical_accuracy: 0.9225 - 54_categorical_accuracy: 0.9441 - val_loss: 4.9405 - val_03_loss: 0.6374 - val_04_loss: 0.1833 - val_13_loss: 0.7783 - val_14_loss: 0.7121 - val_23_loss: 0.0000e+00 - val_24_loss: 1.3245e-08 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.2841 - val_44_loss: 1.5606 - val_53_loss: 0.29

Epoch 19/1000
697/697 [==============================] - 16s 24ms/step - loss: 3.2270 - 03_loss: 0.5352 - 04_loss: 0.0823 - 13_loss: 0.6934 - 14_loss: 0.5675 - 23_loss: 5.9003e-10 - 24_loss: 9.8806e-10 - 33_loss: 6.1009e-10 - 34_loss: 7.3987e-10 - 43_loss: 0.2179 - 44_loss: 0.7849 - 53_loss: 0.2201 - 54_loss: 0.1257 - 03_categorical_accuracy: 0.7243 - 04_categorical_accuracy: 0.9729 - 13_categorical_accuracy: 0.6717 - 14_categorical_accuracy: 0.7037 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9289 - 44_categorical_accuracy: 0.6548 - 53_categorical_accuracy: 0.9279 - 54_categorical_accuracy: 0.9515 - val_loss: 5.4896 - val_03_loss: 0.6924 - val_04_loss: 0.2118 - val_13_loss: 0.8204 - val_14_loss: 0.7711 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.3202 - val_44_loss: 1.7568 - val_53_loss: 0.32

Epoch 25/1000
697/697 [==============================] - 17s 24ms/step - loss: 2.9590 - 03_loss: 0.4950 - 04_loss: 0.0742 - 13_loss: 0.6478 - 14_loss: 0.5380 - 23_loss: 1.7962e-09 - 24_loss: 2.3795e-09 - 33_loss: 2.0216e-09 - 34_loss: 2.1748e-09 - 43_loss: 0.1915 - 44_loss: 0.7111 - 53_loss: 0.1933 - 54_loss: 0.1081 - 03_categorical_accuracy: 0.7482 - 04_categorical_accuracy: 0.9744 - 13_categorical_accuracy: 0.6890 - 14_categorical_accuracy: 0.7193 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9338 - 44_categorical_accuracy: 0.6859 - 53_categorical_accuracy: 0.9332 - 54_categorical_accuracy: 0.9548 - val_loss: 5.9889 - val_03_loss: 0.7594 - val_04_loss: 0.2366 - val_13_loss: 0.8893 - val_14_loss: 0.8190 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.3650 - val_44_loss: 1.8814 - val_53_loss: 0.36

...

Epoch 169/1000
697/697 [==============================] - 16s 23ms/step - loss: 1.6696 - 03_loss: 0.2848 - 04_loss: 0.0563 - 13_loss: 0.3299 - 14_loss: 0.2924 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0990 - 44_loss: 0.4403 - 53_loss: 0.0998 - 54_loss: 0.0671 - 03_categorical_accuracy: 0.8459 - 04_categorical_accuracy: 0.9789 - 13_categorical_accuracy: 0.8281 - 14_categorical_accuracy: 0.8399 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9539 - 44_categorical_accuracy: 0.7780 - 53_categorical_accuracy: 0.9533 - 54_categorical_accuracy: 0.9637 - val_loss: 10.3398 - val_03_loss: 1.2187 - val_04_loss: 0.4106 - val_13_loss: 1.6781 - val_14_loss: 1.4924 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5977 - val_44_loss: 3.1269 - val_53_loss: 0.

Epoch 175/1000
697/697 [==============================] - 16s 23ms/step - loss: 1.6605 - 03_loss: 0.2834 - 04_loss: 0.0561 - 13_loss: 0.3271 - 14_loss: 0.2908 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0986 - 44_loss: 0.4390 - 53_loss: 0.0989 - 54_loss: 0.0666 - 03_categorical_accuracy: 0.8458 - 04_categorical_accuracy: 0.9789 - 13_categorical_accuracy: 0.8286 - 14_categorical_accuracy: 0.8404 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9537 - 44_categorical_accuracy: 0.7788 - 53_categorical_accuracy: 0.9538 - 54_categorical_accuracy: 0.9640 - val_loss: 10.4106 - val_03_loss: 1.2508 - val_04_loss: 0.4100 - val_13_loss: 1.6888 - val_14_loss: 1.5113 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5896 - val_44_loss: 3.1466 - val_53_loss: 0.

Epoch 181/1000
697/697 [==============================] - 16s 23ms/step - loss: 1.6524 - 03_loss: 0.2818 - 04_loss: 0.0562 - 13_loss: 0.3261 - 14_loss: 0.2896 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0979 - 44_loss: 0.4357 - 53_loss: 0.0985 - 54_loss: 0.0666 - 03_categorical_accuracy: 0.8473 - 04_categorical_accuracy: 0.9789 - 13_categorical_accuracy: 0.8292 - 14_categorical_accuracy: 0.8406 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9542 - 44_categorical_accuracy: 0.7801 - 53_categorical_accuracy: 0.9538 - 54_categorical_accuracy: 0.9641 - val_loss: 10.6028 - val_03_loss: 1.2481 - val_04_loss: 0.4045 - val_13_loss: 1.7103 - val_14_loss: 1.5330 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5931 - val_44_loss: 3.2200 - val_53_loss: 0.

Epoch 187/1000
697/697 [==============================] - 17s 24ms/step - loss: 1.6436 - 03_loss: 0.2806 - 04_loss: 0.0559 - 13_loss: 0.3243 - 14_loss: 0.2868 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0975 - 44_loss: 0.4341 - 53_loss: 0.0984 - 54_loss: 0.0660 - 03_categorical_accuracy: 0.8462 - 04_categorical_accuracy: 0.9788 - 13_categorical_accuracy: 0.8292 - 14_categorical_accuracy: 0.8413 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9541 - 44_categorical_accuracy: 0.7807 - 53_categorical_accuracy: 0.9538 - 54_categorical_accuracy: 0.9642 - val_loss: 10.7346 - val_03_loss: 1.2458 - val_04_loss: 0.4301 - val_13_loss: 1.7058 - val_14_loss: 1.5301 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5967 - val_44_loss: 3.2698 - val_53_loss: 0.

KeyboardInterrupt: 

In [31]:
model.save_weights('NextStateModel_MulitLabel_rnn')

In [16]:
labels, encoding, counts = np.unique(rewards, return_inverse=True, return_counts=True)
reward_encoding = np.eye(labels.shape[0])[encoding]
index_to_reward = {}; reward_to_index = {}
for i in range(labels.shape[0]):
    index_to_reward[i] = labels[i]
    reward_to_index[labels[i]] = i
reward_map

{0: -14.100000381469727,
 1: -14.0,
 2: -13.100000381469727,
 3: -13.0,
 4: -12.100000381469727,
 5: -12.0,
 6: -11.0,
 7: -4.099999904632568,
 8: -4.0,
 9: -3.0999999046325684,
 10: -3.0,
 11: -2.0999999046325684,
 12: -2.0,
 13: -1.100000023841858,
 14: -1.0,
 15: -0.10000000149011612,
 16: 0.0}

In [13]:
reward_map[6]

-11.0

In [17]:
np.vectorize(reward_to_index.get)(rewards)

array([16, 16, 15, ..., 13, 13, 15])

In [6]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input, GRU
from keras.layers import Bidirectional
from keras.models import Model

NUM_NODES = 6
NODE_CLASSES = [3, 4]

data_map = {}
losses = []
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        data_map[str(i)+str(n)] = next_states[:,index:index+n]
        losses.append(tf.keras.losses.CategoricalCrossentropy())
        index += n
data_map['reward'] = reward_encoding
losses.append(tf.keras.losses.CategoricalCrossentropy())
        
input_ = Input(shape=(10,42+20,))
#x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(input_)
#x = Bidirectional(GRU(64))(input_)
x = Bidirectional(LSTM(256))(input_)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)

outs = []
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        x_ = Dense(256, activation='relu')(x)
        outs.append(Dense(n, activation='softmax', name=str(i)+str(n))(x_))
        
x_ = Dense(256, activation='relu')(x)
outs.append(Dense(6, activation='softmax', name='reward')(x_))

model = Model(input_, outs)
model.compile(optimizer='adam', loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions_seq_10, data_map, epochs=1000, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=256)

Epoch 1/1000
697/697 [==============================] - 40s 32ms/step - loss: 5.4602 - 03_loss: 0.6317 - 04_loss: 0.1732 - 13_loss: 0.7735 - 14_loss: 0.6846 - 23_loss: 0.0096 - 24_loss: 0.0117 - 33_loss: 0.0084 - 34_loss: 0.0109 - 43_loss: 0.2901 - 44_loss: 1.1840 - 53_loss: 0.2879 - 54_loss: 0.3309 - reward_loss: 1.0636 - 03_categorical_accuracy: 0.6737 - 04_categorical_accuracy: 0.9542 - 13_categorical_accuracy: 0.6534 - 14_categorical_accuracy: 0.6525 - 23_categorical_accuracy: 0.9983 - 24_categorical_accuracy: 0.9984 - 33_categorical_accuracy: 0.9989 - 34_categorical_accuracy: 0.9991 - 43_categorical_accuracy: 0.9178 - 44_categorical_accuracy: 0.4848 - 53_categorical_accuracy: 0.9146 - 54_categorical_accuracy: 0.9051 - reward_categorical_accuracy: 0.5444 - val_loss: 5.3018 - val_03_loss: 0.6191 - val_04_loss: 0.1649 - val_13_loss: 0.7674 - val_14_loss: 0.6652 - val_23_loss: 3.0066e-07 - val_24_loss: 2.0759e-07 - val_33_loss: 1.9189e-07 - val_34_loss: 3.0028e-07 - val_43_loss: 0.270

697/697 [==============================] - 19s 27ms/step - loss: 4.5737 - 03_loss: 0.5893 - 04_loss: 0.1067 - 13_loss: 0.7419 - 14_loss: 0.6122 - 23_loss: 3.1522e-09 - 24_loss: 2.6638e-09 - 33_loss: 2.0591e-09 - 34_loss: 3.2057e-09 - 43_loss: 0.2549 - 44_loss: 0.9402 - 53_loss: 0.2542 - 54_loss: 0.1746 - reward_loss: 0.8996 - 03_categorical_accuracy: 0.6900 - 04_categorical_accuracy: 0.9687 - 13_categorical_accuracy: 0.6605 - 14_categorical_accuracy: 0.6838 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9230 - 44_categorical_accuracy: 0.5879 - 53_categorical_accuracy: 0.9211 - 54_categorical_accuracy: 0.9404 - reward_categorical_accuracy: 0.5934 - val_loss: 5.8639 - val_03_loss: 0.6254 - val_04_loss: 0.1748 - val_13_loss: 0.7712 - val_14_loss: 0.6985 - val_23_loss: 6.0207e-11 - val_24_loss: 1.2041e-11 - val_33_loss: 1.2041e-11 - val_34_loss: 3.0103e-11 - val_43_loss: 0.

697/697 [==============================] - 19s 28ms/step - loss: 3.9384 - 03_loss: 0.5252 - 04_loss: 0.0800 - 13_loss: 0.6841 - 14_loss: 0.5608 - 23_loss: 1.3707e-09 - 24_loss: 1.5828e-09 - 33_loss: 1.2750e-09 - 34_loss: 1.7681e-09 - 43_loss: 0.2117 - 44_loss: 0.7712 - 53_loss: 0.2114 - 54_loss: 0.1167 - reward_loss: 0.7773 - 03_categorical_accuracy: 0.7302 - 04_categorical_accuracy: 0.9736 - 13_categorical_accuracy: 0.6749 - 14_categorical_accuracy: 0.7065 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9303 - 44_categorical_accuracy: 0.6608 - 53_categorical_accuracy: 0.9296 - 54_categorical_accuracy: 0.9531 - reward_categorical_accuracy: 0.6358 - val_loss: 6.8753 - val_03_loss: 0.7028 - val_04_loss: 0.2206 - val_13_loss: 0.8353 - val_14_loss: 0.7795 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.

697/697 [==============================] - 19s 27ms/step - loss: 3.4749 - 03_loss: 0.4629 - 04_loss: 0.0701 - 13_loss: 0.6142 - 14_loss: 0.5152 - 23_loss: 1.0369e-09 - 24_loss: 1.1493e-09 - 33_loss: 9.5260e-10 - 34_loss: 1.2115e-09 - 43_loss: 0.1753 - 44_loss: 0.6742 - 53_loss: 0.1750 - 54_loss: 0.0958 - reward_loss: 0.6922 - 03_categorical_accuracy: 0.7673 - 04_categorical_accuracy: 0.9752 - 13_categorical_accuracy: 0.7056 - 14_categorical_accuracy: 0.7322 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9373 - 44_categorical_accuracy: 0.7011 - 53_categorical_accuracy: 0.9369 - 54_categorical_accuracy: 0.9572 - reward_categorical_accuracy: 0.6708 - val_loss: 7.8959 - val_03_loss: 0.8208 - val_04_loss: 0.2566 - val_13_loss: 0.9469 - val_14_loss: 0.8691 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.

697/697 [==============================] - 18s 26ms/step - loss: 3.1211 - 03_loss: 0.4145 - 04_loss: 0.0655 - 13_loss: 0.5447 - 14_loss: 0.4670 - 23_loss: 2.8364e-10 - 24_loss: 3.7730e-10 - 33_loss: 2.1875e-10 - 34_loss: 3.8465e-10 - 43_loss: 0.1510 - 44_loss: 0.6147 - 53_loss: 0.1505 - 54_loss: 0.0861 - reward_loss: 0.6270 - 03_categorical_accuracy: 0.7935 - 04_categorical_accuracy: 0.9761 - 13_categorical_accuracy: 0.7414 - 14_categorical_accuracy: 0.7613 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9422 - 44_categorical_accuracy: 0.7261 - 53_categorical_accuracy: 0.9423 - 54_categorical_accuracy: 0.9588 - reward_categorical_accuracy: 0.7010 - val_loss: 9.0043 - val_03_loss: 0.9354 - val_04_loss: 0.2774 - val_13_loss: 1.0939 - val_14_loss: 0.9864 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.

...

697/697 [==============================] - 19s 28ms/step - loss: 2.0199 - 03_loss: 0.2772 - 04_loss: 0.0558 - 13_loss: 0.3231 - 14_loss: 0.2857 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0966 - 44_loss: 0.4369 - 53_loss: 0.0969 - 54_loss: 0.0633 - reward_loss: 0.3844 - 03_categorical_accuracy: 0.8474 - 04_categorical_accuracy: 0.9790 - 13_categorical_accuracy: 0.8303 - 14_categorical_accuracy: 0.8413 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9544 - 44_categorical_accuracy: 0.7786 - 53_categorical_accuracy: 0.9545 - 54_categorical_accuracy: 0.9657 - reward_categorical_accuracy: 0.7968 - val_loss: 13.7408 - val_03_loss: 1.2717 - val_04_loss: 0.4385 - val_13_loss: 1.7032 - val_14_loss: 1.5610 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0

697/697 [==============================] - 19s 27ms/step - loss: 2.0092 - 03_loss: 0.2757 - 04_loss: 0.0556 - 13_loss: 0.3208 - 14_loss: 0.2839 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0961 - 44_loss: 0.4355 - 53_loss: 0.0966 - 54_loss: 0.0631 - reward_loss: 0.3820 - 03_categorical_accuracy: 0.8483 - 04_categorical_accuracy: 0.9788 - 13_categorical_accuracy: 0.8300 - 14_categorical_accuracy: 0.8426 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9544 - 44_categorical_accuracy: 0.7792 - 53_categorical_accuracy: 0.9543 - 54_categorical_accuracy: 0.9659 - reward_categorical_accuracy: 0.7969 - val_loss: 13.8567 - val_03_loss: 1.2932 - val_04_loss: 0.4442 - val_13_loss: 1.7580 - val_14_loss: 1.5921 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0

697/697 [==============================] - 19s 28ms/step - loss: 1.9975 - 03_loss: 0.2740 - 04_loss: 0.0554 - 13_loss: 0.3184 - 14_loss: 0.2819 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0957 - 44_loss: 0.4334 - 53_loss: 0.0960 - 54_loss: 0.0629 - reward_loss: 0.3797 - 03_categorical_accuracy: 0.8487 - 04_categorical_accuracy: 0.9790 - 13_categorical_accuracy: 0.8311 - 14_categorical_accuracy: 0.8426 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9545 - 44_categorical_accuracy: 0.7792 - 53_categorical_accuracy: 0.9546 - 54_categorical_accuracy: 0.9660 - reward_categorical_accuracy: 0.7971 - val_loss: 14.2705 - val_03_loss: 1.3124 - val_04_loss: 0.4528 - val_13_loss: 1.7724 - val_14_loss: 1.6216 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0

697/697 [==============================] - 18s 27ms/step - loss: 1.9871 - 03_loss: 0.2727 - 04_loss: 0.0555 - 13_loss: 0.3162 - 14_loss: 0.2799 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0957 - 44_loss: 0.4321 - 53_loss: 0.0953 - 54_loss: 0.0626 - reward_loss: 0.3771 - 03_categorical_accuracy: 0.8487 - 04_categorical_accuracy: 0.9792 - 13_categorical_accuracy: 0.8319 - 14_categorical_accuracy: 0.8433 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9544 - 44_categorical_accuracy: 0.7809 - 53_categorical_accuracy: 0.9547 - 54_categorical_accuracy: 0.9661 - reward_categorical_accuracy: 0.7984 - val_loss: 14.3372 - val_03_loss: 1.3430 - val_04_loss: 0.4488 - val_13_loss: 1.7952 - val_14_loss: 1.6272 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0

KeyboardInterrupt: 

In [ ]:
probs[0]

In [14]:
probs = model.predict(states_actions[0:1])

state = np.zeros(42)
index_state = 0
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        state[index_state+np.random.choice(np.arange(n), p=probs[index][0])] = 1
        index_state += n
        index += 1
state

1/1 [==============================] - 0s 51ms/step


array([0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 1.])

In [42]:
prediction = model.predict(np.array([states_actions[50]]))
print(prediction)

new_state = {}

for i in range(100):
    state = np.zeros(42, dtype=np.int8)
    index_state, index = 0, 0
    for i in range(NUM_NODES):
        for n in NODE_CLASSES:
            state[index_state+np.random.choice(np.arange(n), p=prediction[index][0])] = 1
            index_state += n; index += 1
    if not state.tobytes() in new_state:
        new_state[state.tobytes()] = 1
    else:
        new_state[state.tobytes()] += 1
        
ns = {k: v for k, v in sorted(new_state.items(), key=lambda item: item[1], reverse=True)}
            
for s in ns.keys():
    print(ns[s])
    print(np.frombuffer(s, dtype="int8"))

print(len(new_state.keys()))

1/1 [==============================] - 0s 29ms/step
[array([[9.9999988e-01, 2.8316990e-10, 6.2747894e-08]], dtype=float32), array([[1.4892132e-35, 9.9999988e-01, 3.8243346e-08, 1.2682605e-07]],
      dtype=float32), array([[2.6334348e-08, 6.1443985e-02, 9.3855602e-01]], dtype=float32), array([[0.00808789, 0.7319339 , 0.02746691, 0.23251131]], dtype=float32), array([[1.3898515e-15, 1.0057768e-15, 1.0000000e+00]], dtype=float32), array([[4.0497008e-16, 5.7430210e-16, 4.9967082e-16, 1.0000000e+00]],
      dtype=float32), array([[4.0957807e-16, 3.2186437e-16, 1.0000000e+00]], dtype=float32), array([[7.2782153e-16, 8.9383671e-16, 1.0721520e-15, 1.0000000e+00]],
      dtype=float32), array([[3.632320e-08, 2.473178e-02, 9.752681e-01]], dtype=float32), array([[1.01864416e-07, 3.16646350e-07, 3.63653725e-12, 9.99999523e-01]],
      dtype=float32), array([[2.7840000e-10, 4.2507645e-02, 9.5749241e-01]], dtype=float32), array([[3.4164978e-09, 2.7237392e-13, 1.2582257e-14, 1.0000000e+00]],
      dt

In [38]:
target = states_actions[50]
new_state = {}

for i in range(194040):
    if (target==states_actions[i]).all():
        state = np.array(next_states[i], dtype=np.int8)
        if not state.tobytes() in new_state:
            new_state[state.tobytes()] = 1
        else:
            new_state[state.tobytes()] += 1
        
ns = {k: v for k, v in sorted(new_state.items(), key=lambda item: item[1], reverse=True)}

count = 0
for s in ns.keys():
    print(ns[s])
    count+=ns[s]
    print(np.frombuffer(s, dtype="int8"))

print(len(new_state.keys()))
print(count)

58
[1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
19
[1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
5
[1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1
 0 0 0 0 1]
4
[1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
4
[1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
2
[1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
2
[1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0
 1 0 0 0 1]
1
[1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0
 1 0 0 0 1]
1
[1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
9
96


In [5]:
model.save_weights('Reward_Model_Regression')

In [12]:
rewards

array([ 0. ,  0. ,  0. , ..., -0.1, -0.1, -0.1])

In [21]:
for i in range(rewards.shape[0]):
    if rewards[i] < -10:
        rewards[i] = -11

labels, encoding, counts = np.unique(rewards, return_inverse=True, return_counts=True)
encoding = np.eye(labels.shape[0])[encoding]
reward_map = {}
for i in range(labels.shape[0]):
    reward_map[i] = labels[i]
print(counts)
reward_map

[  3064    134     94   1402   1232   5192   3798  53061   2784 121865
   5374]


{0: -11.0,
 1: -4.099999904632568,
 2: -4.0,
 3: -3.0999999046325684,
 4: -3.0,
 5: -2.0999999046325684,
 6: -2.0,
 7: -1.100000023841858,
 8: -1.0,
 9: -0.10000000149011612,
 10: 0.0}

In [15]:
model = Sequential()
input_ = Input(shape=(10,42+20,))
#x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(input_)
#x = Bidirectional(GRU(64))(input_)
x = Bidirectional(LSTM(128))(input_)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
out = Dense(6, activation='softmax', name='reward')(x)
model = Model(input_, out)

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions_seq_10, encoding, epochs=250, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=256)

Epoch 1/250
697/697 [==============================] - 14s 11ms/step - loss: 1.0678 - categorical_accuracy: 0.5428 - val_loss: 1.0416 - val_categorical_accuracy: 0.5509
Epoch 2/250
697/697 [==============================] - 7s 10ms/step - loss: 1.0319 - categorical_accuracy: 0.5531 - val_loss: 1.0454 - val_categorical_accuracy: 0.5485
Epoch 3/250
697/697 [==============================] - 6s 9ms/step - loss: 1.0206 - categorical_accuracy: 0.5568 - val_loss: 1.0459 - val_categorical_accuracy: 0.5509
Epoch 4/250
697/697 [==============================] - 7s 9ms/step - loss: 1.0084 - categorical_accuracy: 0.5609 - val_loss: 1.0433 - val_categorical_accuracy: 0.5513
Epoch 5/250
697/697 [==============================] - 7s 10ms/step - loss: 0.9960 - categorical_accuracy: 0.5638 - val_loss: 1.0597 - val_categorical_accuracy: 0.5516
Epoch 6/250
697/697 [==============================] - 7s 10ms/step - loss: 0.9836 - categorical_accuracy: 0.5674 - val_loss: 1.0781 - val_categorical_accuracy: 

697/697 [==============================] - 7s 10ms/step - loss: 0.5185 - categorical_accuracy: 0.7536 - val_loss: 2.4186 - val_categorical_accuracy: 0.4695
Epoch 98/250
697/697 [==============================] - 7s 10ms/step - loss: 0.5151 - categorical_accuracy: 0.7554 - val_loss: 2.4796 - val_categorical_accuracy: 0.4640
Epoch 99/250
697/697 [==============================] - 7s 9ms/step - loss: 0.5136 - categorical_accuracy: 0.7556 - val_loss: 2.4406 - val_categorical_accuracy: 0.4617
Epoch 100/250
697/697 [==============================] - 6s 9ms/step - loss: 0.5122 - categorical_accuracy: 0.7558 - val_loss: 2.4977 - val_categorical_accuracy: 0.4727
Epoch 101/250
697/697 [==============================] - 7s 9ms/step - loss: 0.5103 - categorical_accuracy: 0.7570 - val_loss: 2.4595 - val_categorical_accuracy: 0.4703
Epoch 102/250
697/697 [==============================] - 7s 10ms/step - loss: 0.5077 - categorical_accuracy: 0.7587 - val_loss: 2.5412 - val_categorical_accuracy: 0.4758

697/697 [==============================] - 7s 10ms/step - loss: 0.4249 - categorical_accuracy: 0.7880 - val_loss: 3.1004 - val_categorical_accuracy: 0.4638
Epoch 194/250
697/697 [==============================] - 7s 10ms/step - loss: 0.4241 - categorical_accuracy: 0.7888 - val_loss: 3.0625 - val_categorical_accuracy: 0.4619
Epoch 195/250
697/697 [==============================] - 7s 10ms/step - loss: 0.4247 - categorical_accuracy: 0.7868 - val_loss: 3.0508 - val_categorical_accuracy: 0.4658
Epoch 196/250
697/697 [==============================] - 7s 10ms/step - loss: 0.4236 - categorical_accuracy: 0.7878 - val_loss: 3.1272 - val_categorical_accuracy: 0.4614
Epoch 197/250
697/697 [==============================] - 7s 10ms/step - loss: 0.4231 - categorical_accuracy: 0.7888 - val_loss: 3.1508 - val_categorical_accuracy: 0.4579
Epoch 198/250
697/697 [==============================] - 7s 10ms/step - loss: 0.4225 - categorical_accuracy: 0.7893 - val_loss: 3.0730 - val_categorical_accuracy: 0

In [62]:
state_concate = np.concatenate([states_t, next_states_t], axis=1)
state_concate.shape

(198000, 184)

In [63]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input
from keras.layers import Bidirectional
from keras.models import Model


model = Sequential()
model.add(Input(shape=(184,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(11, activation='softmax'))
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=5)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

print(model.summary())

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(state_concate, encoding, epochs=250, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=64)

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_131 (Dense)           (None, 512)               94720     
                                                                 
 dense_132 (Dense)           (None, 512)               262656    
                                                                 
 dense_133 (Dense)           (None, 512)               262656    
                                                                 
 dense_134 (Dense)           (None, 11)                5643      
                                                                 
Total params: 625,675
Trainable params: 625,675
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/250
2785/2785 [==============================] - 16s 5ms/step - loss: 0.4057 - categorical_accuracy: 0.8597 - val_loss: 0.3373 - val_categorical_accuracy: 0.8796
Epoc

In [64]:
model.save_weights('RewardModel')
np.save('reward_map.npy', reward_map) 

In [49]:
np.argmax(encoding[0:1000], axis=-1).shape

(1000,)

In [66]:
y = model.predict(state_concate[0:50000])
confusion = tf.math.confusion_matrix(labels=np.argmax(encoding[0:50000], axis=-1), predictions=np.argmax(y, axis=-1), num_classes=11)
confusion

1563/1563 [==============================] - 3s 2ms/step


<tf.Tensor: shape=(11, 11), dtype=int32, numpy=
array([[  938,     0,     0,     0,    10,     0,     0,     0,     0,
            0,     0],
       [    0,    24,     0,     3,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,    25,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,   259,     0,     8,     0,     1,     0,
            0,     0],
       [    1,     0,     2,     0,   345,     0,    19,     0,     2,
            0,     0],
       [    0,     0,     0,     4,     0,   925,     1,    18,     3,
           18,     0],
       [    0,     0,     0,     0,     9,     0,  1027,     0,    15,
            2,     0],
       [    2,     0,     0,     0,     0,     7,     0, 11347,     4,
         1568,     2],
       [    0,     0,     0,     0,     0,     0,     3,     6,   518,
            2,    16],
       [    0,     0,     0,     0,     0,     0,     5,   356,     7,
        31137,    49],
       [    

In [33]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input
from keras.layers import Bidirectional
from keras.models import Model


model = Sequential()
model.add(Input(shape=(STATE_LEN+1+ACTION_LEN,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(11, activation='softmax'))
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

print(model.summary())

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions_t, encoding, epochs=250, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=64)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_87 (Dense)            (None, 512)               68608     
                                                                 
 dense_88 (Dense)            (None, 512)               262656    
                                                                 
 dense_89 (Dense)            (None, 512)               262656    
                                                                 
 dense_90 (Dense)            (None, 11)                5643      
                                                                 
Total params: 599,563
Trainable params: 599,563
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/250
2785/2785 [==============================] - 18s 6ms/step - loss: 0.4729 - categorical_accuracy: 0.8179 - val_loss: 0.3974 - val_categorical_accuracy: 0.8393
Epoch

KeyboardInterrupt: 

In [25]:
target = states[53]
rewards_ = {}

for i in range(194040):
    if (target==states[i]).all():
        reward = rewards[i]
        if not reward in rewards_:
            rewards_[reward] = 1
        else:
            rewards_[reward] += 1
        
ns = {k: v for k, v in sorted(rewards_.items(), key=lambda item: item[1], reverse=True)}

count = 0
for s in ns.keys():
    print(ns[s])
    count+=ns[s]
    print(s)



11334
-1.0
528
-1.100000023841858
43
-2.0


In [67]:
target = states_actions[53]
rewards_ = {}

for i in range(194040):
    if (target==states_actions[i]).all():
        reward = rewards[i-1]
        if not reward in rewards_:
            rewards_[reward] = 1
        else:
            rewards_[reward] += 1
        
ns = {k: v for k, v in sorted(rewards_.items(), key=lambda item: item[1], reverse=True)}

count = 0
for s in ns.keys():
    print(ns[s])
    count+=ns[s]
    print(s)

print(len(new_state.keys()))
print(count)

59
-1.0
10
0.0
6
-1.100000023841858
5
-2.0
1
80
